<a href="https://colab.research.google.com/github/rskim47/BigDataDistributedProgramming/blob/master/Project_1_Fall_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <p style="text-align: center;">MIS 285N: Big Data and Distributed Programming</p>
# <p style="text-align: center;">Project - 1 : Apache Spark</p>
## <p style="text-align: center;">Instructor: Dr. Ramesh Yerraballi</p>
## <p style="text-align: center;">Due: Tuesday, September 14th submitted via Canvas by 11:59 pm</p>

Your work should be written in a **Jupyter notebook**.   

Also, please make sure your code runs in your notebook before submitting.

**Note:**

This project is based on Map-Reduce Framework. In these you will get to work with Spark and will get to know how 
does spark work, what functionalities does spark provide, what does map-reduce framework do and why is it useful. 

In this project you will be implementing a basic song recommender system. You will be given a dataset where there are multiple csv files. These csv files have data corresponding to song play count and song information.

The data you would be using will be provided in a zip file along with this notebook. The __msd.zip__ archive contains:
1. **'kaggle_visible_evaluation_triplets.txt'**. We will be using the visible part of the testing data to understand the working on Apache Spark.  The user's listening history is provided as: (user, song, play count).  
2. In **'kaggle_songs.txt'** file, each song is marked using an index for easier representation of songs.  
3. And **'kaggle_users.txt'** file is the canonical list of user identifiers.
4. Take **'MSDChallengeGettingstarted.pdf'** as your reference.



### **What to turn in?**  

A zip folder which will have:
1. Jupyter Notebook
2. A brief report in PDF format on what features you used for recommendation. And a brief explanation of flow of your code. For example,  what RDD does what or, why it was created.
3. datasets folder with the csv files you are using in your notebook.
4. Notebook should use relative path to the csv files in datasets folder.
5. Name of the zip folder - `<your_name>_<your_partner_name>.zip`

In [ ]:
# Setting up environment
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
!pip install -q findspark


This project consists of 4 questions:  

1. Create an RDD with _kaggle_visible_evaluation_triplets.txt_ and replace the song name with the song index from _kaggle_songs.txt_. Identify the number of songs that do not have any rating. 
2. Generate song ratings based on the song play count as a normalized score between 0 and 1. 
3. Identify the popular song based on this rating and recommend songs to user, given user id based on the algorithm used in Movie recommender system from class. 
4. Using Cosine similarity function, identify pair-wise similarity between each pair of users and generate the top 5 most similar users without an overlap in users. 

The above list is the higer level idea about the questions. 

In [ ]:
### Starter code ####
import findspark
print(findspark.find())
findspark.init() #May need to change based on machine
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local[*]").setAppName("Songs")
sc = SparkContext(conf = conf)
#### These lines are to tell jupyter where to find Apache Spark ####

In [ ]:
#sc.stop()

In [ ]:
## Read triplet file into RDD
triplet_rdd = sc.textFile(r"kaggle_visible_evaluation_triplets.txt") \
    .map(lambda line: line.split("\t")) 

triplet_rdd.take(5)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Step 1: 
Replace song name with song index and identify the number of songs without user history

In [ ]:
songs_rdd = sc.textFile(r"kaggle_songs.txt") \
    .map(lambda line: line.split()) 

print(songs_rdd.take(5))

#1- Replace Song name with Index 
joined_rdd=triplet_rdd.map(lambda x: (x[1],[x[0],x[2]])).join(songs_rdd)
print(joined_rdd.take(5)) #reordered column
final_rdd=joined_rdd.map(lambda x: x[1])
print(final_rdd.take(5)) 
refinal_rdd=final_rdd.map(lambda x: (x[0][0],int(x[0][1]),x[1]))
print(refinal_rdd.take(5))

#2- Identify Songs without history
nodupes_rdd=refinal_rdd.map(lambda x: x[2])
nodupes_rdd=nodupes_rdd.distinct()
print(nodupes_rdd.take(10))
x=nodupes_rdd.count()
y=songs_rdd.count()
nohistory_songs=y-x
print(nohistory_songs)


## Step 2:
Generate song ratings based on the play_count. For example, if (song_1, 5; song_2, 10; song_3, 5) i.e., song_1 is played 5 times, song_2 is played 10 times and song_3 is played 5 times, the normalized rating score should be 0.25, 0.5 and 0.25 respectively. 
Similarly, generate the rating for all the songs. You may notice that based on all songs, the rating is almost always very low. So, think of the best way to convert song count to ratings. (Hint: Try generating ratings based on each user's song play history)

In [ ]:
#1- Play counts
plays_rdd=refinal_rdd.map(lambda x: (x[2],x[1]))
plays_rdd=plays_rdd.reduceByKey(lambda x,y: x+y)
print(plays_rdd.take(5))

#2- Ratings
user_rdd=refinal_rdd.map(lambda x: (x[0],x[1]))
user_rdd=user_rdd.reduceByKey(lambda x,y: x+y)
#print(user_rdd.take(5))

ratings_rdd=refinal_rdd.map(lambda x: (x[0],[x[1],x[2]])).join(user_rdd)
#print(ratings_rdd.take(5))
ratings_rdd=ratings_rdd.map(lambda x: (x[0],x[1][0][0],x[1][0][1],x[1][1]))

ratings_rdd=ratings_rdd.map(lambda x: (x[2],x[1]/x[3]))
#print(ratings_rdd.take(5))

total_rdd=ratings_rdd.groupByKey().map(lambda x : (x[0], list(x[1])))
print(total_rdd.take(5))
overallratings_rdd = total_rdd.map(lambda x : (x[0],sum(x[1]) / len(x[1])))
print(overallratings_rdd.take(5))

## Step 3: 
For a given user_id (choose one by yourselves), rating, recommend 5 other songs from the list. One way to do this is based on another user who liked the same song liked by this user with rating more than the given rating and recommend the 5 songs based on the matched user's rating. 

## Step 4: 
1. Compute cosine similarity between all pairs of users. 
2. Sort the similarity score and print the top-5 similar users. 
3. If the top-5 user set has an user appearing more than once, ignore that pair and take the next best pair from the sorted list. 
4. For a given user_id, identify the top-5 similar users and hence song recommendations from other user's list. 